In [1]:
# import the necessary packages
import numpy as np
import random
import time
import cv2
import os

In [2]:
# load the COCO class labels the Mask R-CNN was trained on
labelsPath = os.path.sep.join(['mask-rcnn-coco', "object_detection_classes_coco.txt"])
LABELS = open(labelsPath).read().strip().split("\n")

In [3]:
# load the set of colors that will be used when visualizing a given instance segmentation
# base path to mask-rcnn directory
colorsPath = os.path.sep.join(['mask-rcnn-coco', "colors.txt"])
COLORS = open(colorsPath).read().strip().split("\n")
COLORS = [np.array(c.split(",")).astype("int") for c in COLORS]
COLORS = np.array(COLORS, dtype="uint8")

In [4]:
# derive the paths to the Mask R-CNN weights and model configuration
weightsPath = os.path.sep.join(['mask-rcnn-coco', "frozen_inference_graph.pb"])
configPath = os.path.sep.join(['mask-rcnn-coco', "mask_rcnn_inception_v2_coco_2018_01_28.pbtxt"])

In [5]:
# load our Mask R-CNN trained on the COCO dataset (90 classes) from disk
print("[INFO] loading Mask R-CNN from disk...")
net = cv2.dnn.readNetFromTensorflow(weightsPath, configPath)

[INFO] loading Mask R-CNN from disk...


In [6]:
# load our input image and grab its dimensions
image = cv2.imread('C:/Users/Ron/Projects/openCV/mask_rcnn/traffic.jpg')
(H, W) = image.shape[:2]

In [7]:
# construct a blob from the input image and then perform a forward
# pass of the Mask R-CNN, giving us (1) the bounding box  coordinates
# of the objects in the image along with (2) the pixel-wise segmentation for each specific object
blob = cv2.dnn.blobFromImage(image, swapRB=True, crop=False)
net.setInput(blob)
start = time.time()
(boxes, masks) = net.forward(["detection_out_final", "detection_masks"])
end = time.time()

# show timing information and volume information on Mask R-CNN
print("Completed Mask R-CNN in {:.6f} seconds".format(end - start))

Completed Mask R-CNN in 11.603995 seconds


In [8]:
# clone our original image so we can draw on it
clone = image.copy()
# loop over the number of detected objects
for i in range(0, boxes.shape[2]):
	# extract the class ID of the detection along with the confidence
	# (i.e., probability) associated with the prediction
	classID = int(boxes[0, 0, i, 1])
	confidence = boxes[0, 0, i, 2]

	# filter out weak predictions by ensuring the detected probability
	# is greater than the minimum probability
	# type=float, default=0.5
	if confidence > 0.5:

		# scale the bounding box coordinates back relative to the
		# size of the image and then compute the width and the height of the bounding box
		box = boxes[0, 0, i, 3:7] * np.array([W, H, W, H])
		(startX, startY, endX, endY) = box.astype("int")
		boxW = endX - startX
		boxH = endY - startY

		# extract the pixel-wise segmentation for the object, resize
		# the mask such that it's the same dimensions of the bounding
		# box, and then finally threshold to create a *binary* mask
		mask = masks[i, classID]
		mask = cv2.resize(mask, (boxW, boxH), interpolation=cv2.INTER_NEAREST)
		#minimum threshold for pixel-wise mask segmentation type=float, default=0.3
		mask = (mask > 0.3)

		# extract the ROI of the image
		roi = clone[startY:endY, startX:endX]

		# now, extract *only* the masked region of the ROI by passing
		# in the boolean mask array as our slice condition
		roi = roi[mask]

		# randomly select a color that will be used to visualize this
		# particular instance segmentation then create a transparent
		# overlay by blending the randomly selected color with the ROI
		color = random.choice(COLORS)
		blended = ((0.4 * color) + (0.6 * roi)).astype("uint8")

		# store the blended ROI in the original image
		clone[startY:endY, startX:endX][mask] = blended

		# draw the bounding box of the instance on the image
		color = [int(c) for c in color]
		cv2.rectangle(clone, (startX, startY), (endX, endY), color, 2)

		# draw the predicted label and associated probability of the
		# instance segmentation on the image
		text = "{}: {:.4f}".format(LABELS[classID], confidence)
		cv2.putText(clone, text, (startX, startY - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

In [9]:
# show the output image
cv2.imshow("Output", clone)
cv2.waitKey(0)

-1